In [ ]:
# 코랩기준 우선 설치 후 런타인 다시시작 해야됨
!pip install "scikit_learn==0.22.2.post1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 14.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 2.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
import os
import numpy as np

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
class CFG:
    SEED = 42

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

## Data Load

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/X_train_fine.csv')
X_test = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/X_test_fine.csv')
y = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/y_train_fine.csv')
submit = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/sample_submission.csv')
submit = submit.iloc[:,:1]
submit_proba = submit.copy()
y_train = y['class']

### Model_Baseline

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
# Models
models = [GaussianNB(),
          LabelSpreading(),
          NearestCentroid(),
          LabelPropagation(),
          RidgeClassifierCV(),
          KNeighborsClassifier(),
          QuadraticDiscriminantAnalysis(),
          MLPClassifier(random_state=CFG.SEED),
          RidgeClassifier(random_state=CFG.SEED),
          ExtraTreeClassifier(random_state=CFG.SEED),
          ExtraTreesClassifier(random_state=CFG.SEED),
          DecisionTreeClassifier(random_state=CFG.SEED),
          RandomForestClassifier(random_state=CFG.SEED),
          LinearSVC(multi_class="crammer_singer",random_state=CFG.SEED),
          # One-Vs-One
          SVC(random_state=CFG.SEED),
          NuSVC(random_state=CFG.SEED),
          GaussianProcessClassifier(multi_class="one_vs_one",random_state=CFG.SEED),
          # One-Vs-The-Rest
          Perceptron(random_state=CFG.SEED),
          SGDClassifier(random_state=CFG.SEED),
          GradientBoostingClassifier(random_state=CFG.SEED),
          PassiveAggressiveClassifier(random_state=CFG.SEED),
          LinearSVC(multi_class="ovr",random_state=CFG.SEED),
          GaussianProcessClassifier(multi_class="one_vs_rest",random_state=CFG.SEED),
          LogisticRegression(multi_class="ovr",solver='liblinear',random_state=CFG.SEED),
          LogisticRegressionCV(multi_class="ovr",solver='liblinear',random_state=CFG.SEED)]
# Models_name
models_name = list(map(str, models))
for i in range(len(models_name)):
  models_name[i] = models_name[i].split('(')[0]

In [ ]:
new_models = []
for j in [5,10,15]:
  print(f'----- n_splits={j} -----')
  for i in range(len(models)):
    skf = StratifiedKFold(n_splits=j, shuffle=True, random_state=CFG.SEED)    
    cv_score = cross_val_score(models[i], X_train, y_train, scoring='f1_macro', cv=skf).mean()
    if cv_score >= 0.96:
      new_models.append(models[i])
    print(round(cv_score, 3),':',models_name[i])

----- n_splits=5 -----
0.95 : GaussianNB
0.974 : LabelSpreading
0.95 : NearestCentroid
0.974 : LabelPropagation
0.968 : RidgeClassifierCV
0.971 : KNeighborsClassifier
0.93 : QuadraticDiscriminantAnalysis
0.98 : MLPClassifier
0.971 : RidgeClassifier
0.933 : ExtraTreeClassifier
0.974 : ExtraTreesClassifier
0.962 : DecisionTreeClassifier
0.968 : RandomForestClassifier
0.965 : LinearSVC
0.971 : SVC
0.962 : NuSVC
0.974 : GaussianProcessClassifier
0.974 : Perceptron
0.977 : SGDClassifier
0.974 : GradientBoostingClassifier
0.971 : PassiveAggressiveClassifier
0.971 : LinearSVC
0.974 : GaussianProcessClassifier
0.968 : LogisticRegression
0.962 : LogisticRegressionCV
----- n_splits=10 -----
0.961 : GaussianNB
0.974 : LabelSpreading
0.953 : NearestCentroid
0.974 : LabelPropagation
0.974 : RidgeClassifierCV
0.968 : KNeighborsClassifier
0.939 : QuadraticDiscriminantAnalysis
0.982 : MLPClassifier
0.977 : RidgeClassifier
0.956 : ExtraTreeClassifier
0.971 : ExtraTreesClassifier
0.944 : DecisionTreeCla

In [ ]:
new_models = list(set(new_models))
new_models_name = list(map(str, models))
for i in range(len(new_models_name)):
  new_models_name[i] = new_models_name[i].split('(')[0]

In [ ]:
submit_proba[['A','B','C']] = 0
for i in range(len(new_models)):
  new_models[i].fit(X_train,y_train)
  submit[f'{new_models_name[i]}'] = new_models[i].predict(X_test)
  try:
    submit_proba[['A','B','C']] += np.round(new_models[i].predict_proba(X_test),3)
  except:
    pass

In [ ]:
ABC = submit.iloc[:,1:].apply(pd.value_counts, axis=1).fillna(0)
ABC['rate'] = ABC.max(axis=1)/ABC.sum(axis=1)

In [ ]:
ABC.sort_values(by='rate',axis=0).head(20)

,A,B,C,rate
90,0.0,9.0,12.0,0.571429
117,0.0,12.0,9.0,0.571429
60,13.0,0.0,8.0,0.619048
19,0.0,13.0,8.0,0.619048
119,0.0,4.0,17.0,0.809524
162,0.0,17.0,4.0,0.809524
84,0.0,18.0,3.0,0.857143
126,0.0,18.0,3.0,0.857143
83,0.0,19.0,2.0,0.904762
35,0.0,19.0,2.0,0.904762


In [ ]:
submit_proba = submit_proba.iloc[:,1:]/12

In [ ]:
submit_proba.loc[103]

A    0.035917
B    0.068167
C    0.895917
Name: 103, dtype: float64

In [ ]:
# fit함수에 sample_weight는 다 있음 -> class weight와 비슷하다고함 일괄적으로 적용가능 -> 적용/미적용 버전 생성가능
# Boosting계열 없음(추가예정)